In [99]:
import pandas as pd
import numpy as np
import pickle
import time
import os

In [100]:
prefix = './TempData/Linux_Process_Detect/'
dirnames = [prefix+name+'/' for name in os.listdir(prefix) if not name.startswith('.') and os.path.isdir(prefix+name)]

In [103]:
dirnames

['./TempData/Linux_Process_Detect/30_training_set_cpu/',
 './TempData/Linux_Process_Detect/5_training_set_cpu/',
 './TempData/Linux_Process_Detect/22_training_set_usb/',
 './TempData/Linux_Process_Detect/17_training_set_usb/',
 './TempData/Linux_Process_Detect/14_training_set_wifi/',
 './TempData/Linux_Process_Detect/26_training_set_ok/',
 './TempData/Linux_Process_Detect/11_training_set_wifi/',
 './TempData/Linux_Process_Detect/1_training_set_cpu/',
 './TempData/Linux_Process_Detect/18_training_set_usb/',
 './TempData/Linux_Process_Detect/23_training_set_usb/',
 './TempData/Linux_Process_Detect/4_training_set_cpu/',
 './TempData/Linux_Process_Detect/27_training_set_ok/',
 './TempData/Linux_Process_Detect/10_training_set_cpu/',
 './TempData/Linux_Process_Detect/31_training_set_cpu/',
 './TempData/Linux_Process_Detect/16_training_set_usb/',
 './TempData/Linux_Process_Detect/12_training_set_wifi/',
 './TempData/Linux_Process_Detect/13_training_set_wifi/',
 './TempData/Linux_Process_Detec

In [104]:
names = []
for dirname in dirnames:
    names.extend([dirname+name for name in os.listdir(dirname) if not name.startswith('.')])

In [105]:
names

['./TempData/Linux_Process_Detect/30_training_set_cpu/wer18_response.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer23_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer14_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer2_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer19_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer19_response.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer7_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer11_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer26_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer22_response.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer16_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer21_normal.csv',
 './TempData/Linux_Process_Detect/30_training_set_cpu/wer7_response.csv',
 './TempData/Linux_Process_Detect/30_training_

In [106]:
dataset = pd.DataFrame()

In [107]:
dataset = pd.DataFrame()
content = []
for name in names:
    temp = pd.read_csv(name, index_col=False, header=None)
    content.append(temp)
dataset = pd.concat(content, ignore_index=True)

In [108]:
# dataset = pd.concat(content, ignore_index=True)
dataset.shape

(2166, 40185)

In [109]:
q=dataset.values
np.random.shuffle(q)
labels = q[:,0]
datas = q[:,1:] 
orders = np.argsort(datas)[:,::-1]
big_q = np.sort(datas)[:,::-1]

In [ ]:
cut_len = 20
orders_arr = orders[:,:cut_len]
big_q_arr = big_q[:,:cut_len]
pairs_arr = [(np.concatenate((orders_arr[i][np.newaxis, :], big_q_arr[i][np.newaxis, :]), axis=0),
              labels[i]) for i in range(len(orders_arr))]

In [ ]:
# Save pairs to pickle file

s=time.time()
pickle.dump(pairs_arr, open('./Datasets/train_pairs_%d.pkl'%cut_len,'wb+'))
print("cost:%.4f"%(time.time()-s))

In [140]:
# len(set(np.array(pairs_100)[:,1]))
# len(pairs_50[0][0][0])
len(pairs_arr)

2166

### Statistics of data

In [141]:
print("Raw data shape:",datas.shape)
print("Arranged data shape:(",len(pairs_arr),',',len(pairs_arr[0][0][0]),')')
print("Labels:",set(labels.astype(int))," Labels number:",len(set(labels)))
print("Train data number:",int(np.floor(datas.shape[0]*0.7)),", Test data number:",int(np.ceil(datas.shape[0]*0.3)))

Raw data shape: (2166, 40184)
Arranged data shape:( 2166 , 100 )
Labels: {0, 1, 2, 3}  Labels number: 4
Train data number: 1516 , Test data number: 650


In [36]:
s=time.time()
pickle.dump(q, open('./TempData/Linux_Process_Detect/training_set_all.pkl','wb+'))
print("cost:%.4f"%(time.time()-s))

cost:1.2401


In [20]:
s=time.time()
q = pickle.load(open('./TempData/Linux_Process_Detect/training_set_all.pkl','rb+'))
print("cost:%.4f"%(time.time()-s))

cost:0.4172


In [30]:
set(q[:,0])

{0.0, 2.0, 3.0}

In [31]:
num = 0
for item in q:
    if item[0]==0:
        num+=1
print(num,num/742)

678


In [37]:
w=np.concatenate((q,np.zeros((len(q),776))),axis=1)
s=time.time()
pickle.dump(w, open('./Datasets/ts_all_padded.pkl','wb+'))
print("cost:%.4f"%(time.time()-s))

cost:1.1774


dataset.to_csv('./TempData/Linux_Process_Detect/training_set_all.csv', header=False, index=False)

s=time.time()
dataset = pd.read_csv('./TempData/Linux_Process_Detect/training_set_all.csv', index_col=False, header=None)
print("cost:%.4f"%(time.time()-s))

In [120]:
# def load_data(opt, root='./Datasets/training_set_all.pkl'):
start = time.time()
all_data = q
print("==> Load data successfully, time elapsed: %.4f" % (time.time() - start))
length = len(pairs_100)
# opt.NUM_CLASSES = len(set(all_data[:, 0]))
# all_pairs = [(i[1:], i[0]) for i in all_data]
sep_point = int(np.floor(0.7*length))
train_pairs = pairs_100[:sep_point]
test_pairs = pairs_100[sep_point:]
# return train_pairs, test_pairs

==> Load data successfully, time elapsed: 0.0019


In [127]:
num = [0]*4
for item in pairs_100:
    if item[1]==0:
        num[0]+=1
    elif item[1]==1:
        num[1]+=1
    elif item[1]==2:
        num[2]+=1
    elif item[1]==3:
        num[3]+=1
# print(num,len(pairs_100),len(pairs_100)-num,num/len(pairs_100))

In [128]:
num

[1975, 104, 18, 69]

### Data Information:

In [ ]:
Labels:      0     1   2   3
All_Data:   [1975, 104, 18, 69]
Train_Data: [1371, 81 , 13, 51]
Test_Data:  [604 , 23 , 5 , 18]

In [ ]:
Labels:      0     1   2   3
All_Data:   [1083, 87, 14, 50]
Train_Data: [758 , 61, 11, 33]
Test_Data:  [325 , 26, 3 , 17]

### Test Result: 

In [ ]:
Labels:                  0     1   2   3
Right_when_testing_all: [1945, 85, 8 , 57]
Right_of_training_set : [1356, 76, 7 , 51]
Right_of_testing_set  : [589 , 9 , 1 , 6 ]

In [ ]:
Labels:                  0     1   2   3
Right_when_testing_all: [1060, 70, 10, 36]
Right_of_training_set : [743 , 54, 10, 33]
Right_of_testing_set  : [317 , 16, 0 , 3 ]

In [25]:
def divPrime(num):
    lt = []
    print(num, '=', end=' ')
    while num != 1:
        for i in range(2, int(num+1)):
            if num % i == 0:  # i是num的一个质因数
                lt.append(i)
                num = num / i # 将num除以i，剩下的部分继续分解
                break
    for i in range(0, len(lt)-1):
        print(lt[i], '*', end=' ')
 
    print(lt[-1])

In [26]:
divPrime(40184+8+256+512)
divPrime(40960)

40960 = 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 5
40960 = 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 5


In [27]:
8+256+512

776

(length + 2*pad - size)/stride + 1 
N = (W − F + 2P )/S+1
N = (W-16+8)/8+1=W/8-1
W = (N-1)*S + F -2P
W = (N-1)*8 + 16 = 8N+8

In [174]:
40960 / 8**4

10.0

In [173]:
num = 0
div = 40960
for i in range(4):
    if (div-16) % 8 ==0:
        div = (div-16)//8 + 1
        continue
    else:
        div = (div-16)//8 + 2
        num += ((div-16)%8)*8**(i+1)
    print(div)
        

766
95
11
